<a href="https://colab.research.google.com/github/surabhi13gupta/LangChains/blob/main/json_output_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_huggingface
!pip install langchain_core

In [3]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

In [4]:
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)
model = ChatHuggingFace(llm=llm)

In [8]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

In [6]:
parser = JsonOutputParser()

In [9]:
template = PromptTemplate(
    template = """Give me the name, age and city of fictional person \n {format_instruction}""",
    input_variables = [],
    partial_variables = {'format_instruction': parser.get_format_instructions()}
)

In [15]:
template.format()

'Give me the name, age and city of fictional person \n Return a JSON object.'

In [16]:
prompt = template.format()
result = model.invoke(prompt)
final_result = parser.parse(result.content)
print(final_result)

{'name': 'Alice Bright', 'age': 28, 'city': 'Aspenwood'}


##Using Langchain

In [19]:
chain = template | model | parser
result = chain.invoke({})
print(result)

{'name': 'Anya Petrova', 'age': 27, 'city': 'Berlin, Germany'}


## Pass topic dynamically

In [20]:
template = PromptTemplate(
    template = """Give me the 5 facts about {topic} \n {format_instruction}""",
    input_variables = ["topic"],
    partial_variables = {'format_instruction': parser.get_format_instructions()}
)

In [21]:
chains = template | model | parser

In [24]:
result = chains.invoke({"topic": "black hole"})

In [26]:
from pprint import pprint
pprint(result)

{'facts': ['**Black holes are regions of spacetime with such strong gravity '
           'that nothing, not even light, can escape them.**',
           '**They form when massive stars collapse at the end of their '
           'lives.**',
           '**Their gravitational pull warps the fabric of spacetime around '
           'them, creating a distortion zone known as the event horizon.**',
           '**Mass is concentrated at a singular point called the singularity, '
           'which is believed to be infinitely dense.**',
           '**Black holes come in various sizes, from stellar-mass black holes '
           'to supermassive black holes at the centers of galaxies.**']}


##Above we can see that schema is not enforced - list of facts but not mentioned like fact1, fact2 and so on